
# How to make the perfect time-lapse of the Earth

This tutorial shows a detail coverage of making time-lapse animations from satellite imagery like a pro.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#0.-Prerequisites" data-toc-modified-id="0.-Prerequisites-1">0. Prerequisites</a></span></li><li><span><a href="#1.-Removing-clouds" data-toc-modified-id="1.-Removing-clouds-2">1. Removing clouds</a></span></li><li><span><a href="#2.-Applying-co-registration" data-toc-modified-id="2.-Applying-co-registration-3">2. Applying co-registration</a></span></li><li><span><a href="#3.-Large-Area-Example" data-toc-modified-id="3.-Large-Area-Example-4">3. Large Area Example</a></span></li><li><span><a href="#4.-Split-Image" data-toc-modified-id="4.-Split-Image-5">4. Split Image</a></span></li></ul></div>

Note: This notebook requires an installation of additional packages `ffmpeg-python` and `ipyleaflet`.

In [ ]:
%load_ext autoreload
%autoreload 2

import datetime as dt
import json
import os
import subprocess
from concurrent.futures import ProcessPoolExecutor
from datetime import date, datetime, time, timedelta
from functools import partial
from glob import glob

import ffmpeg
import geopandas as gpd
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely
from ipyleaflet import GeoJSON, Map, basemaps
from shapely.geometry import Polygon
from tqdm.auto import tqdm

from eolearn.core import (EOExecutor, EOPatch, EOTask, FeatureType,
                          LinearWorkflow, LoadTask, OverwritePermission,
                          SaveTask, ZipFeatureTask)
from eolearn.coregistration import ECCRegistration
from eolearn.features import LinearInterpolation, SimpleFilterTask
from eolearn.io import ExportToTiff, ImportFromTiff, SentinelHubInputTask
from eolearn.mask import CloudMaskTask

from sentinelhub import (CRS, BatchSplitter, BBox, BBoxSplitter,
                         DataCollection, Geometry, MimeType, SentinelHubBatch,
                         SentinelHubRequest, SHConfig, bbox_to_dimensions)


## 0. Prerequisites

In order to set everything up and make the credentials work, please check [this notebook](https://github.com/sentinel-hub/eo-learn/blob/master/examples/io/SentinelHubIO.ipynb).

In [ ]:
class AnimateTask(EOTask):
    def __init__(self, image_dir, out_dir, out_name, feature=(FeatureType.DATA, 'RGB'), scale_factor=2.5, duration=3, dpi=150, pad_inches=None, shape=None):
        self.image_dir = image_dir
        self.out_name = out_name
        self.out_dir = out_dir
        self.feature = feature
        self.scale_factor = scale_factor
        self.duration = duration
        self.dpi = dpi
        self.pad_inches = pad_inches
        self.shape = shape
        
    def execute(self, eopatch):
        images = np.clip(eopatch[self.feature]*self.scale_factor, 0, 1)
        fps = len(images)/self.duration
        subprocess.run(f'rm -rf {self.image_dir} && mkdir {self.image_dir}', shell=True)
        
        for idx, image in enumerate(images):
            if self.shape:
                fig = plt.figure(figsize=(self.shape[0], self.shape[1]))
            plt.imshow(image)
            plt.axis(False)
            plt.savefig(f'{self.image_dir}/image_{idx:03d}.png', bbox_inches='tight', dpi=self.dpi, pad_inches = self.pad_inches)
            plt.close()
        
        # video related
        stream = ffmpeg.input(f'{self.image_dir}/image_*.png', pattern_type='glob', framerate=fps)
        stream = stream.filter('pad', w='ceil(iw/2)*2', h='ceil(ih/2)*2', color='white')
        split = stream.split()
        video = split[0]
        
        # gif related
        palette = split[1].filter('palettegen', reserve_transparent=True, stats_mode='diff')
        gif = ffmpeg.filter([split[2], palette], 'paletteuse', dither='bayer', bayer_scale=5, diff_mode='rectangle')
        
        # save output
        os.makedirs(self.out_dir, exist_ok=True)
        video.output(f'{self.out_dir}/{self.out_name}.mp4', crf=15, pix_fmt='yuv420p', vcodec='libx264', an=None).run(overwrite_output=True)
        gif.output(f'{self.out_dir}/{self.out_name}.gif').run(overwrite_output=True)
        return eopatch

## 1. Removing clouds

In [ ]:
# https://twitter.com/Valtzen/status/1270269337061019648
bbox = BBox(bbox=[-73.558102,45.447728,-73.488750,45.491908], crs=CRS.WGS84)
resolution = 10
time_interval = ('2018-01-01', '2020-01-01')
print(f'Image size: {bbox_to_dimensions(bbox, resolution)}')

geom, crs = bbox.geometry, bbox.crs
wgs84_geometry = Geometry(geom, crs).transform(CRS.WGS84)
geometry_center = wgs84_geometry.geometry.centroid

map1 = Map(
    basemap=basemaps.Esri.WorldImagery,
    center=(geometry_center.y, geometry_center.x),
    zoom=13
)

area_geojson = GeoJSON(data=wgs84_geometry.geojson)
map1.add_layer(area_geojson)

map1

In [ ]:
download_task = SentinelHubInputTask(
    bands = ['B04', 'B03', 'B02'],
    bands_feature = (FeatureType.DATA, 'RGB'),
    resolution=resolution,
    maxcc=0.9,
    time_difference=timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L2A,
    max_threads=10,
    mosaicking_order='leastCC',
    additional_data=[
        (FeatureType.MASK, 'CLM'),
        (FeatureType.MASK, 'dataMask')
    ]
)

def valid_coverage_thresholder_f(valid_mask, more_than=0.95):
    coverage = np.count_nonzero(valid_mask)/np.prod(valid_mask.shape)
    return coverage > more_than

valid_mask_task = ZipFeatureTask({FeatureType.MASK: ['CLM', 'dataMask']}, (FeatureType.MASK, 'VALID_DATA'),
                                 lambda clm, dm: np.all([clm == 0, dm], axis=0))

filter_task = SimpleFilterTask((FeatureType.MASK, 'VALID_DATA'), valid_coverage_thresholder_f)

name = 'clm_service'
anim_task = AnimateTask(image_dir = './images', out_dir = './animations', out_name=name, duration=5, dpi=200)

params = {'MaxIters': 500}
coreg_task = ECCRegistration((FeatureType.DATA, 'RGB'), channel=2, params=params)

name = 'clm_service_coreg'
anim_task_after = AnimateTask(image_dir='./images', out_dir='./animations', out_name=name, duration=5, dpi=200)

In [ ]:
workflow = LinearWorkflow(
    download_task,
    valid_mask_task,
    filter_task,
    anim_task,
    coreg_task,
    anim_task_after
)

result = workflow.execute({
    download_task: {'bbox': bbox, 'time_interval': time_interval}
})

## 2. Applying co-registration

In [ ]:
bbox = BBox(bbox=[34.716, 30.950, 34.743, 30.975], crs=CRS.WGS84)
resolution = 10
time_interval = ('2020-01-01', '2021-01-01')
print(f'BBox size: {bbox_to_dimensions(bbox, resolution)}')

geom, crs = bbox.geometry, bbox.crs
wgs84_geometry = Geometry(geom, crs).transform(CRS.WGS84)
geometry_center = wgs84_geometry.geometry.centroid

map1 = Map(
    basemap=basemaps.Esri.WorldImagery,
    center=(geometry_center.y, geometry_center.x),
    zoom=14
)

area_geojson = GeoJSON(data=wgs84_geometry.geojson)
map1.add_layer(area_geojson)

map1

In [ ]:
download_task_l2a = SentinelHubInputTask(
    bands = ['B04', 'B03', 'B02'],
    bands_feature = (FeatureType.DATA, 'RGB'),
    resolution=resolution,
    maxcc=0.9,
    time_difference=timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L2A,
    max_threads=10,
    additional_data=[
        (FeatureType.MASK, 'dataMask', 'dataMask_l2a')
    ]
)

download_task_l1c = SentinelHubInputTask(
    bands_feature = (FeatureType.DATA, 'BANDS'),
    resolution=resolution,
    maxcc=0.9,
    time_difference=timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L1C,
    max_threads=10,
    additional_data=[
        (FeatureType.MASK, 'dataMask', 'dataMask_l1c')
    ]
)

data_mask_merge = ZipFeatureTask({FeatureType.MASK: ['dataMask_l1c', 'dataMask_l2a']}, (FeatureType.MASK, 'dataMask'),
                                 lambda dm1, dm2: np.all([dm1, dm2], axis=0))

cloud_masking_task = CloudMaskTask(
    data_feature=(FeatureType.DATA, 'BANDS'),
    is_data_feature='dataMask',
    all_bands=True,
    processing_resolution=120,
    mono_features=None,
    mask_feature='CLM',
    average_over=16,
    dilation_size=12,
    mono_threshold=0.2
)

valid_mask_task = ZipFeatureTask({FeatureType.MASK: ['CLM', 'dataMask']}, (FeatureType.MASK, 'VALID_DATA'),
                                 lambda clm, dm: np.all([clm == 0, dm], axis=0))

filter_task = SimpleFilterTask((FeatureType.MASK, 'VALID_DATA'), valid_coverage_thresholder_f)

name = 'wo_coreg_anim'
anim_task_before = AnimateTask(image_dir='./images', out_dir='./animations', out_name=name, duration=5, dpi=200)


params = {'MaxIters': 500}
coreg_task = ECCRegistration((FeatureType.DATA, 'RGB'), channel=2, params=params)

name = 'coreg_anim'
anim_task_after = AnimateTask(image_dir='./images', out_dir='./animations', out_name=name, duration=5, dpi=200)

In [ ]:
workflow = LinearWorkflow(
    download_task_l2a,
    download_task_l1c,
    data_mask_merge,
    cloud_masking_task,
    valid_mask_task,
    filter_task,
    anim_task_before,
    coreg_task,
    anim_task_after
)

result = workflow.execute({
    download_task_l2a: {'bbox': bbox, 'time_interval': time_interval}
})

## 3. Large Area Example

In [ ]:
bbox = BBox(bbox=[21.4,-20.0,23.9,-18.0], crs=CRS.WGS84)
time_interval = ('2017-09-01', '2019-04-01')
# time_interval = ('2017-09-01', '2017-10-01')
resolution = 640
print(f'BBox size: {bbox_to_dimensions(bbox, resolution)}')

geom, crs = bbox.geometry, bbox.crs
wgs84_geometry = Geometry(geom, crs).transform(CRS.WGS84)
geometry_center = wgs84_geometry.geometry.centroid

map1 = Map(
    basemap=basemaps.Esri.WorldImagery,
    center=(geometry_center.y, geometry_center.x),
    zoom=8
)

area_geojson = GeoJSON(data=wgs84_geometry.geojson)
map1.add_layer(area_geojson)

map1

In [ ]:
download_task_l2a = SentinelHubInputTask(
    bands = ['B04', 'B03', 'B02'],
    bands_feature = (FeatureType.DATA, 'RGB'),
    resolution=resolution,
    maxcc=0.9,
    time_difference=timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L2A,
    max_threads=10,
    additional_data=[
        (FeatureType.MASK, 'dataMask', 'dataMask_l2a')
    ],
    aux_request_args={'dataFilter': {'previewMode': 'PREVIEW'}}
)

download_task_l1c = SentinelHubInputTask(
    bands_feature = (FeatureType.DATA, 'BANDS'),
    resolution=resolution,
    maxcc=0.9,
    time_difference=timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L1C,
    max_threads=10,
    additional_data=[
        (FeatureType.MASK, 'dataMask', 'dataMask_l1c')
    ],
    aux_request_args={'dataFilter': {'previewMode': 'PREVIEW'}}
)

data_mask_merge = ZipFeatureTask({FeatureType.MASK: ['dataMask_l1c', 'dataMask_l2a']}, (FeatureType.MASK, 'dataMask'),
                                 lambda dm1, dm2: np.all([dm1, dm2], axis=0))

cloud_masking_task = CloudMaskTask(
    data_feature='BANDS',
    is_data_feature='dataMask',
    all_bands=True,
    processing_resolution=resolution,
    mono_features=('CLP', 'CLM'),
    mask_feature=None,
    mono_threshold=0.3,
    average_over=1,
    dilation_size=4
)

valid_mask_task = ZipFeatureTask({FeatureType.MASK: ['CLM', 'dataMask']}, (FeatureType.MASK, 'VALID_DATA'),
                                 lambda clm, dm: np.all([clm == 0, dm], axis=0))

resampled_range = ('2018-01-01', '2019-01-01', 10)
interp_task = LinearInterpolation(
    feature=(FeatureType.DATA, 'RGB'),
    mask_feature=(FeatureType.MASK, 'VALID_DATA'),
    resample_range=resampled_range,
    bounds_error=False
)

name = 'botswana_single_raw'
anim_task_raw = AnimateTask(image_dir='./images', out_dir='./animations', out_name=name, duration=5, dpi=200)

name = 'botswana_single'
anim_task = AnimateTask(image_dir='./images', out_dir='./animations', out_name=name, duration=3, dpi=200)

In [ ]:
workflow = LinearWorkflow(
    download_task_l2a,
#     anim_task_raw
    download_task_l1c,
    data_mask_merge,
    cloud_masking_task,
    valid_mask_task,
    interp_task,
    anim_task
)

result = workflow.execute({
    download_task_l2a:{'bbox': bbox, 'time_interval': time_interval},
})

## 4. Split Image

In [ ]:
bbox = BBox(bbox=[21.3,-20.0,24.0,-18.0], crs=CRS.WGS84)
time_interval = ('2018-09-01', '2020-04-01')
resolution = 120

bbox_splitter = BBoxSplitter([bbox.geometry], bbox.crs, (6,5))
bbox_list = np.array(bbox_splitter.get_bbox_list())
info_list = np.array(bbox_splitter.get_info_list())
print(f'{len(bbox_list)} patches of size: {bbox_to_dimensions(bbox_list[0], resolution)}')

gdf = gpd.GeoDataFrame(None, crs=int(bbox.crs.epsg), geometry=[bbox.geometry for bbox in bbox_list])

geom, crs = gdf.unary_union, CRS.WGS84
wgs84_geometry = Geometry(geom, crs).transform(CRS.WGS84)
geometry_center = wgs84_geometry.geometry.centroid

map1 = Map(
    basemap=basemaps.Esri.WorldImagery,
    center=(geometry_center.y, geometry_center.x),
    zoom=8
)

for geo in gdf.geometry:
    area_geojson = GeoJSON(data=Geometry(geo, crs).geojson)
    map1.add_layer(area_geojson)

map1

In [ ]:
download_task = SentinelHubInputTask(
    bands = ['B04', 'B03', 'B02'],
    bands_feature = (FeatureType.DATA, 'RGB'),
    resolution=resolution,
    maxcc=0.9,
    time_difference=timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L2A,
    max_threads=10,
    additional_data=[
        (FeatureType.MASK, 'CLM'),
        (FeatureType.DATA, 'CLP'),
        (FeatureType.MASK, 'dataMask')
    ]
)

valid_mask_task = ZipFeatureTask([(FeatureType.MASK, 'dataMask'), (FeatureType.MASK, 'CLM'), (FeatureType.DATA, 'CLP')], (FeatureType.MASK, 'VALID_DATA'),
                                 lambda dm, clm, clp: np.all([dm, clm == 0, clp/255 < 0.3], axis=0))

resampled_range = ('2019-01-01', '2020-01-01', 10)
interp_task = LinearInterpolation(
    feature=(FeatureType.DATA, 'RGB'),
    mask_feature=(FeatureType.MASK, 'VALID_DATA'),
    resample_range=resampled_range,
    bounds_error=False
)

export_r = ExportToTiff(feature=(FeatureType.DATA, 'RGB'), folder='./tiffs/', band_indices=[0])
export_g = ExportToTiff(feature=(FeatureType.DATA, 'RGB'), folder='./tiffs/', band_indices=[1])
export_b = ExportToTiff(feature=(FeatureType.DATA, 'RGB'), folder='./tiffs/', band_indices=[2])

convert_to_uint16 = ZipFeatureTask([(FeatureType.DATA, 'RGB')], (FeatureType.DATA, 'RGB'),
                                   lambda x: (x*1e4).astype(np.uint16))

In [ ]:
os.system('rm -rf ./tiffs && mkdir ./tiffs')

workflow = LinearWorkflow(
    download_task,
    valid_mask_task,
    interp_task,
    convert_to_uint16,
    export_r,
    export_g,
    export_b
)

# Execute the workflow
execution_args = []
for idx, bbox in enumerate(bbox_list):
    execution_args.append({
        download_task: {'bbox': bbox, 'time_interval': time_interval},
        export_r: {'filename': f'r_patch_{idx}.tiff'},
        export_g: {'filename': f'g_patch_{idx}.tiff'},
        export_b: {'filename': f'b_patch_{idx}.tiff'}
    })
                      
executor = EOExecutor(workflow, execution_args, save_logs=True)
executor.run(workers=10, multiprocess=False)
executor.make_report()

In [ ]:
# spatial merge
subprocess.run(f'gdal_merge.py -n 0 -a_nodata 0 -o tiffs/r.tiff -co compress=LZW tiffs/r_patch_*.tiff && rm -rf tiffs/r_patch_*.tiff', shell=True);
subprocess.run(f'gdal_merge.py -n 0 -a_nodata 0 -o tiffs/g.tiff -co compress=LZW tiffs/g_patch_*.tiff && rm -rf tiffs/g_patch_*.tiff', shell=True);
subprocess.run(f'gdal_merge.py -n 0 -a_nodata 0 -o tiffs/b.tiff -co compress=LZW tiffs/b_patch_*.tiff && rm -rf tiffs/b_patch_*.tiff', shell=True);

In [ ]:
dates = pd.date_range('2019-01-01', '2020-01-01', freq='10D').to_pydatetime()
import_r = ImportFromTiff((FeatureType.DATA, 'R'), f'tiffs/r.tiff', timestamp_size=len(dates))
import_g = ImportFromTiff((FeatureType.DATA, 'G'), f'tiffs/g.tiff', timestamp_size=len(dates))
import_b = ImportFromTiff((FeatureType.DATA, 'B'), f'tiffs/b.tiff', timestamp_size=len(dates))

merge_bands_task = ZipFeatureTask({FeatureType.DATA: ['R', 'G', 'B']}, (FeatureType.DATA, 'RGB'),
                                  lambda r, g, b: np.moveaxis(np.array([r[...,0], g[...,0], b[...,0]]), 0, -1))
                                   
def temporal_ma_f(f):
    k = np.array([0.05, 0.6, 1, 0.6, 0.05])
    k = k/np.sum(k)
    w = len(k)//2
    return np.array([np.sum([f[(i-w+j)%len(f)]*k[j] for j in range(len(k))], axis=0) for i in range(len(f))])

temporal_smoothing = ZipFeatureTask([(FeatureType.DATA, 'RGB')], (FeatureType.DATA, 'RGB'), temporal_ma_f)

name = 'botswana_multi_ma'
anim_task = AnimateTask(image_dir='./images', out_dir='./animations', out_name=name, duration=3,
                        dpi=400, scale_factor=3.0/1e4)

In [ ]:
workflow = LinearWorkflow(
    import_r,
    import_g,
    import_b,
    merge_bands_task,
    temporal_smoothing,
    anim_task
)

result = workflow.execute()

## 5. Batch request

Use the evalscript from the [custom scripts repository](https://github.com/sentinel-hub/custom-scripts/tree/master/sentinel-2/interpolated_time_series) and see how to use it in the batch example in our [sentinelhub-py](https://github.com/sentinel-hub/sentinelhub-py/blob/master/examples/batch_processing.ipynb) library.